Test automated DMD alignment

In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
from matplotlib.widgets import Button
import numpy as np
from skimage.io import imread
import json
import sys
plt.ion()

In [2]:
calibration_image_fname = r'Z:/davis/data/dmd_calibration/20170705/white.png'
calibration_image = imread(calibration_image_fname)
calibration_data_fname = r'C:\Users\Ahrenslab\Documents\Github\dmd\dmd_calibration.json'

with open(calibration_data_fname, 'r') as infile:
    calibration_data = json.load(infile)

In [3]:
calibration_data

{'dmd_rect': [181, 593, 1812, 1484]}

In [4]:
template_image = np.zeros([2048, 2048])
template_pattern = np.zeros([480, 854]) + 30000
corner_pos = calibration_data['dmd_rect']
template_image[corner_pos[1]:corner_pos[1] + template_pattern.shape[0], corner_pos[0]:corner_pos[0]+template_pattern.shape[1]] = template_pattern

In [5]:
fig, axs = plt.subplots(ncols=2, figsize=(16,8))
axs[0].imshow(calibration_image)
axs[1].imshow(template_image)

In [6]:
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform2D,
                                   RigidTransform2D,
                                   AffineTransform2D)

In [10]:
static = calibration_image.astype('float32')
moving = template_image.astype('float32')
g2w = np.eye(3)
g2w[:2,-1] = np.array(static.shape) / 2

In [8]:
plt.imshow(moving + static)

In [11]:
c_of_mass = transform_centers_of_mass(static, g2w,
                                      moving, g2w)

In [12]:
nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)
level_iters = [10000, 1000, 100]
sigmas = [3.0, 1.0, 0.0]
factors = [4, 2, 1]
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

transform = TranslationTransform2D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(static, moving, transform, params0,
                              g2w, g2w,
                              starting_affine=starting_affine)

Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]


In [13]:
transform = AffineTransform2D()
params0 = None
starting_affine = translation.affine
affine = affreg.optimize(static, moving, transform, params0,
                         g2w, g2w,
                         starting_affine=starting_affine)

Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]


In [14]:
plt.imshow(affine.transform(moving))